# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 12:38PM,240535,21,619130,"NBTY Global, Inc.",Released
1,Jun 27 2022 12:37PM,240533,10,0085799939,ISDIN Corporation,Released
2,Jun 27 2022 12:37PM,240533,10,0085799938,ISDIN Corporation,Released
3,Jun 27 2022 12:37PM,240533,10,0085799944,ISDIN Corporation,Released
4,Jun 27 2022 12:37PM,240533,10,0085799946,ISDIN Corporation,Released
5,Jun 27 2022 12:37PM,240533,10,0085799951,ISDIN Corporation,Released
6,Jun 27 2022 12:37PM,240533,10,0085804098,ISDIN Corporation,Released
7,Jun 27 2022 12:37PM,240533,10,0085804108,ISDIN Corporation,Released
8,Jun 27 2022 12:37PM,240533,10,0085804113,ISDIN Corporation,Released
9,Jun 27 2022 12:37PM,240533,10,0085804232,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,240531,Released,1
44,240532,Released,1
45,240533,Released,50
46,240534,Released,1
47,240535,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240531,NaN,1.0
240532,NaN,1.0
240533,NaN,50.0
240534,NaN,1.0
240535,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240455,0.0,4.0
240467,0.0,49.0
240470,0.0,50.0
240471,0.0,57.0
240475,5.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240455,0,4
240467,0,49
240470,0,50
240471,0,57
240475,5,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240455,0,4
1,240467,0,49
2,240470,0,50
3,240471,0,57
4,240475,5,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240455,,4
1,240467,,49
2,240470,,50
3,240471,,57
4,240475,5,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc."
1,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation
51,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc."
52,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc."
53,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc."
54,Jun 27 2022 12:13PM,240522,10,ISDIN Corporation
64,Jun 27 2022 12:13PM,240514,10,ISDIN Corporation
104,Jun 27 2022 12:12PM,240530,10,Bio-PRF
105,Jun 27 2022 11:52AM,240527,20,"ACI Healthcare USA, Inc."
118,Jun 27 2022 11:51AM,240529,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",,1
1,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,,50
2,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",,1
3,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",,1
4,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",,1
5,Jun 27 2022 12:13PM,240522,10,ISDIN Corporation,,10
6,Jun 27 2022 12:13PM,240514,10,ISDIN Corporation,,40
7,Jun 27 2022 12:12PM,240530,10,Bio-PRF,,1
8,Jun 27 2022 11:52AM,240527,20,"ACI Healthcare USA, Inc.",,13
9,Jun 27 2022 11:51AM,240529,10,"Methapharm, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1,
1,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,50,
2,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",1,
3,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",1,
4,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",1,
5,Jun 27 2022 12:13PM,240522,10,ISDIN Corporation,10,
6,Jun 27 2022 12:13PM,240514,10,ISDIN Corporation,40,
7,Jun 27 2022 12:12PM,240530,10,Bio-PRF,1,
8,Jun 27 2022 11:52AM,240527,20,"ACI Healthcare USA, Inc.",13,
9,Jun 27 2022 11:51AM,240529,10,"Methapharm, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1,
1,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,50,
2,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",1,
3,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",1,
4,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1.0,NaN
1,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,50.0,NaN
2,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",1.0,NaN
3,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",1.0,NaN
4,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1.0,0.0
1,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,50.0,0.0
2,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",1.0,0.0
3,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",1.0,0.0
4,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3367119,288.0,2.0
12,240505,1.0,1.0
15,1202510,17.0,46.0
16,480999,2.0,0.0
19,721475,5.0,24.0
20,1202533,65.0,6.0
21,1683561,5.0,2.0
22,481056,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3367119,288.0,2.0
1,12,240505,1.0,1.0
2,15,1202510,17.0,46.0
3,16,480999,2.0,0.0
4,19,721475,5.0,24.0
5,20,1202533,65.0,6.0
6,21,1683561,5.0,2.0
7,22,481056,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,288.0,2.0
1,12,1.0,1.0
2,15,17.0,46.0
3,16,2.0,0.0
4,19,5.0,24.0
5,20,65.0,6.0
6,21,5.0,2.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,288.0
1,12,Released,1.0
2,15,Released,17.0
3,16,Released,2.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,2.0,1.0,46.0,0.0,24.0,6.0,2.0,0.0
Released,288.0,1.0,17.0,2.0,5.0,65.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,2.0,1.0,46.0,0.0,24.0,6.0,2.0,0.0
1,Released,288.0,1.0,17.0,2.0,5.0,65.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,2.0,1.0,46.0,0.0,24.0,6.0,2.0,0.0
1,Released,288.0,1.0,17.0,2.0,5.0,65.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()